In [1]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/kaggle"

In [ ]:
%cd /content/drive/MyDrive/kaggle/

In [ ]:
!kaggle datasets download -d muratkokludataset/grapevine-leaves-image-dataset

In [ ]:
!kaggle datasets download -d muratkokludataset/grapevine-leaves-image-dataset --unzip --force


In [ ]:
from PIL import Image
image1 = Image.open("/content/drive/MyDrive/kaggle/Grapevine_Leaves_Image_Dataset/Ak/Ak (1).png")
image1

## Splitting

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
data = ImageDataGenerator(rescale = 1/255.0,validation_split=0.2) # herevalidadation split means test split

# trainData = data.flow_from_directory(directory = "/content/drive/MyDrive/kaggle/Grapevine_Leaves_Image_Dataset",
#                                      target_size=(256, 256),class_mode='categorical', batch_size=32,shuffle=True, subset = "training")
# testData = data.flow_from_directory(directory = "/content/drive/MyDrive/kaggle/Grapevine_Leaves_Image_Dataset",
#                                     target_size=(256, 256),class_mode='categorical', batch_size=32,shuffle=True, subset = "validation")

## Data Generators


In [ ]:
train_gen = ImageDataGenerator(rescale = 1./255., 
                              rotation_range = 40 , 
                              shear_range = .2 , 
                              width_shift_range = .2,
                              height_shift_range = .2 , 
                              horizontal_flip=True,
                              vertical_flip= True ,
                               zoom_range=0.2,
                              fill_mode='nearest')

In [ ]:
train_d_gen = train_gen.flow_from_directory(train_dir , class_mode = 'categorical' ,
                                            target_size = (150 , 150) , batch_size = 16)

In [ ]:
val_gen = ImageDataGenerator(rescale = 1./255.)

In [ ]:
val_d_gen = val_gen.flow_from_directory(validation_dir , class_mode = 'categorical' ,
                                        target_size = (150 , 150) , batch_size = 2)

## Modeling

In [ ]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32 , (3,3) , activation = 'relu' , input_shape = (150 , 150 , 3)),
    tf.keras.layers.MaxPooling2D(2,2) , 
    tf.keras.layers.Conv2D(64 , (3,3) , activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128 , (3,3) , activation = 'relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512 , activation = 'relu'),
    tf.keras.layers.Dense(5,activation = 'softmax')
])

model.summary()

In [ ]:
model.compile(loss = tf.keras.losses.categorical_crossentropy,
              optimizer = tf.keras.optimizers.Adam() , metrics = ['accuracy'])

In [ ]:
history = model.fit(train_d_gen , validation_data=val_d_gen , epochs = 20)

## Overfitting check

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize = (12 , 6))

plt.plot(loss , 'b' , label = 'training loss')
plt.plot(val_loss , 'orange' , label = 'validation loss')

plt.legend()
plt.show()


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(acc , 'b' , label = 'training acc')
plt.plot(val_acc , 'orange' , label = 'val acc')

plt.legend()
plt.show()

## Evaluating

In [ ]:
model.evaluate(val_d_gen)

## Transfer learning

In [ ]:
#I will use  InceptionV3
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
inception_v3 = InceptionV3(include_top= False , weights= None , input_shape = (150 , 150 , 3))

In [ ]:
!wget --no-check-certificate https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 

In [ ]:
inception_v3.load_weights('inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [ ]:
#freezing the layers so that they are not affected by the training on the new dataset
for layer in inception_v3.layers:
    layer.trainable = False

In [ ]:
inception_v3.summary()

## Transfer learning _ overfitting check

In [ ]:
loss = history_pre_trained.history['loss']
val_loss = history_pre_trained.history['val_loss']
acc = history_pre_trained.history['accuracy']
val_acc = history_pre_trained.history['val_accuracy']

plt.figure(figsize = (12 , 6))
plt.plot(loss , 'b' , label = 'training loss')
plt.plot(val_loss , 'orange' , label = 'validation loss')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize = (12 , 6))
plt.plot(acc , 'b' , label = 'training accuracy')
plt.plot(val_acc , 'orange' , label = 'validation accuracy')
plt.legend()
plt.show()